In [10]:
import numpy as np 
import pandas as pd 
import pymysql

In [11]:
# This method takes the database connection.
conn = pymysql.connect(host='localhost', port=int(3306), user='yabaconsultoria', passwd='yaba2389', db="livedwine",
                           charset='utf8')



# Build dataframe
df = pd.read_sql_query(
        "SELECT wine.wine_id, alcohol_content, country, harvest, wine_name, producer, region, service, type, volume, (SELECT GROUP_CONCAT(wine_grape.grape separator ', ') from wine_grape where wine_id = wine.wine_id) as grape, (SELECT GROUP_CONCAT(wine_harmonization.harmonization separator ', ') from wine_harmonization where wine_id = wine.wine_id) as harmonization, rating.rating, user.user_id, user_name, age, profession, gender FROM wine INNER JOIN rating on wine.wine_id = rating.wine_id INNER JOIN user on rating.user_id = user.user_id",
        conn)

conn.close()

df.drop(columns=['type', 'country', 'region', 'alcohol_content', 'producer', 'service', 'volume', 'grape', 'harvest', 'harmonization', 'user_name', 'gender', 'profession', 'age'], axis=1, inplace=True)
df.head()


,wine_id,wine_name,rating,user_id
0,1,Abadia del Roble White La Mancha D.O,4.0,2
1,1,Abadia del Roble White La Mancha D.O,4.0,3
2,1,Abadia del Roble White La Mancha D.O,2.0,4
3,1,Abadia del Roble White La Mancha D.O,2.0,5
4,1,Abadia del Roble White La Mancha D.O,3.0,6


In [12]:
# Set wine's ratings and total ratings.
wines_sizes_ratings = df.groupby('wine_id').agg({'rating': [np.size, np.mean]})
sizes = wines_sizes_ratings['rating']['size']
means = wines_sizes_ratings['rating']['mean']
df.drop_duplicates('wine_id', inplace=True)
for i in range(len(wines_sizes_ratings)):
    df['vote_count'] = sizes.values
    df['vote_average'] = means.values
    
# Displaying the ordered dataframe
df.drop(columns=['user_id', 'rating'], axis=1, inplace=True)
df.sort_values(['vote_average', 'vote_count'], ascending=False).head()

,wine_id,wine_name,vote_count,vote_average
62695,117,Frei Brothers Reserve Chardonnay 2017,546.0,3.163004
17270,33,Canepa Finisimo Gran Reserva Sauvignon Blanc 2014,534.0,3.159176
28070,53,Domaine Le Mourre Châteauneuf-du-Pape AOC 2016,546.0,3.157509
59978,112,Portada Winemaker's Selection 2017,556.0,3.140288
18339,35,Castillo D'Elaro Bobal 2018,560.0,3.119643


In [13]:
# Mean of average of the dataframe.
c = df['vote_average'].mean()
print(c)

3.0079311152400967


In [14]:
# Now we calculate the 'm' to execute algorithm. -> m: minimum number of votes required to be listed
m = df['vote_count'].quantile(0.75)
print(m)

549.0


In [15]:
# Qualified wines, based on vote counts.
q_wines = df.copy().loc[df['vote_count'] >= m]
q_wines.shape


(45, 4)

In [16]:
def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    R = x['vote_average']
    # Calculation based on the IMDB formula
    return (v/(v+m) * R) + (m/(m+v) * c)

In [17]:
# Define a new feature 'score' and calculate its value with `weighted_rating()
q_wines['score'] = q_wines.apply(weighted_rating, axis=1)
q_wines = q_wines.sort_values('score', ascending=False)
q_wines.head()

,wine_id,wine_name,vote_count,vote_average,score
59978,112,Portada Winemaker's Selection 2017,556.0,3.140288,3.074529
18339,35,Castillo D'Elaro Bobal 2018,560.0,3.119643,3.064341
88812,165,La Combe D Or Grenache 2018,556.0,3.113309,3.060954
3284,7,Anciano Reserva 7 Years Old Magnum 2011,549.0,3.112933,3.060432
2186,5,Anciano Crianza 3 years Tempranillo Valdepeñas...,549.0,3.091075,3.049503
